In [ ]:
import os
import dspy

In [ ]:
lm = dspy.LM(
    model="groq/openai/gpt-oss-20b",
    api_key=os.environ.get("GROQ_API_KEY"),
    base_url="https://api.groq.com/openai/v1",
)

dspy.configure(lm=lm)

In [ ]:
predict = dspy.Predict("question -> answer")

prediction = predict(
    question="Which is the best Large Language Model ever created?")
print(prediction.answer)

In [ ]:
lm.inspect_history(3)

In [ ]:
class QA(dspy.Signature):
    question = dspy.InputField()
    answer = dspy.OutputField()


predict = dspy.Predict(QA)
prediction = predict(question="Who is the creator of GNU?")

prediction.answer

In [ ]:
class QA(dspy.Signature):
    """Given the question, generate the answer"""
    question = dspy.InputField(desc="User's question.")
    answer = dspy.OutputField(desc="Often between 1 to 20 words.")


predict = dspy.Predict(QA)
prediction = predict(question="Why was GNU created?")

prediction.answer

In [ ]:
lm.inspect_history(1)

In [ ]:
cot = dspy.ChainOfThought(QA)
prediction = cot(question="Is Linux a full-fledged operating system if yes then what is GNU?")

In [ ]:
prediction.reasoning

In [ ]:
prediction.answer

In [ ]:
lm.inspect_history(1)

In [ ]:
class QA1(dspy.Signature):
    question = dspy.InputField(desc="Users question")
    step_by_step_thought = dspy.OutputField(desc="Step-by-step thought in 50 to 80 words")


class QA2(dspy.Signature):
    question = dspy.InputField(desc="Users question")
    thought = dspy.InputField(desc="Your step-by-step thoughts")
    final_concise_response = dspy.OutputField(
        desc="Answer in 5 to 7 words")

In [ ]:
class DoubleChainOfThoughtModule(dspy.Module):
    def __init__(self):
        self.cot1 = dspy.ChainOfThought(QA1)
        self.cot2 = dspy.ChainOfThought(QA2)

    def forward(self, question):
        thought = self.cot1(question=question).step_by_step_thought
        final_concise_response = self.cot2(question=question,
                                           thought=thought).final_concise_response
        return dspy.Prediction(thought=thought, final_concise_response=final_concise_response)

In [ ]:
multi_step_thinking = "Which operating system was created by the creator of the the programming language used by Google in modern backend development"

doubleCot = DoubleChainOfThoughtModule()
output = doubleCot(question=multi_step_thinking)

In [ ]:
output.final_concise_response

In [ ]:
lm.inspect_history(2)

In [ ]:
from pydantic import BaseModel,Field

class AnswerConfidence(BaseModel):
    answer: str = Field("Answer in 1 to 5 words.")
    confidence: float = Field("Your confidence between 0 to 1.")

class QAWithConfidence(dspy.Signature):
    question = dspy.InputField(desc="Users question.")
    answer: AnswerConfidence = dspy.OutputField(desc="Response in 7 to 15 words.")

In [ ]:
predict = dspy.ChainOfThought(QAWithConfidence)

question = "Who has the most contributions in the life of an average person. Ritchie, Stallman, Torvalds or Thompson?"

output = predict(question=question)

In [ ]:
output

In [ ]:
output.reasoning

In [ ]:
output.answer.answer

In [ ]:
output.answer.confidence

In [ ]:
class AnswerElement(BaseModel):
    rank: int = Field(
        "Rank that language in today's programming scene 1 to n - rank MUST BE UNIQUE")
    programming_language: str = Field("Name of the programming language")
    created_at: int = Field("The year this language was created")
    peak_popularity: int = Field("Year this language was most popular in")
    use_case: list[str] = Field(
        "Most important usecases of this language - atmost 3")


class QAList(dspy.Signature):
    question = dspy.InputField()
    output: list[AnswerElement] = dspy.OutputField()

In [ ]:
question = "40 programming languages"

predict = dspy.ChainOfThought(QAList)
output = predict(question=question)

In [ ]:
languages = output.output

languages

In [ ]:
languages.__len__()